# 【問題1】ブレンディングのスクラッチ実装
ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を 最低3つ 示してください。精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを指します。


ブレンディングとは
ブレンディングとは、N個の多様なモデルを独立して学習させ、推定結果を重み付けした上で足し合わせる方法です。最も単純には平均をとります。多様なモデルとは、以下のような条件を変化させることで作り出すものです。


手法（例：線形回帰、SVM、決定木、ニューラルネットワークなど）
ハイパーパラメータ（例：SVMのカーネルの種類、重みの初期値など）
入力データの前処理の仕方（例：標準化、対数変換、PCAなど）

重要なのはそれぞれのモデルが大きく異なることです。


回帰問題でのブレンディングは非常に単純であるため、scikit-learnには用意されていません。


《補足》


分類問題の場合は、多数決を行います。回帰問題に比べると複雑なため、scikit-learnにはVotingClassifierが用意されています。

In [1]:
# データインポート
import pandas as pd
from sklearn.model_selection import train_test_split


dir_path = "/Users/yuki.tatsuoka/Downloads/house-prices-advanced-regression-techniques (1)/"

df = pd.read_csv(dir_path + "train.csv")

# 説明変数と目的変数を抽出
X = df.loc[:, ['GrLivArea','YearBuilt']]
y = df['SalePrice']

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

scaler = StandardScaler()
X_scaler = scaler.fit_transform(X)

# 訓練用とテスト用に分割する
X_train, X_test, y_train, y_test = train_test_split(X_scaler, y, test_size=0.2, random_state=42)

# 重回帰モデル
model_linear = LinearRegression().fit(X_train, y_train)
pred_linear = model_linear.predict(X_test)
print("LinearRegresssion MSE {}".format(mean_squared_error(pred_linear, y_test)))

# SVM
model_svm = SVR().fit(X_train, y_train)
pred_svm = model_svm.predict(X_test)
print("SVR MSE {}".format(mean_squared_error(pred_svm, y_test)))

# 決定木
model_tree = DecisionTreeRegressor().fit(X_train, y_train)
pred_tree = model_tree.predict(X_test)
print("決定木 MSE {}".format(mean_squared_error(pred_tree, y_test)))

LinearRegresssion MSE 2495554898.668321
SVR MSE 7842006843.379719
決定木 MSE 2187354209.919045


In [3]:
p =0.5
p2 = 0.5

# ブレンディング：重回帰、svm
pred = pred_linear*p + pred_svm *p2
print("重回帰、svm MSE {}".format(mean_squared_error(pred, y_test)))

# ブレンディング：重回帰、決定木
pred = pred_linear*p + pred_tree *p2
print("重回帰、決定木 MSE {}".format(mean_squared_error(pred, y_test)))

# ブレンディング：svm, 決定木
pred = pred_svm*p + pred_tree *p2
print("svm、決定木 MSE {}".format(mean_squared_error(pred, y_test)))

#  ブレンディング：svm, 決定木、重回帰
pred = pred_svm*p + pred_tree *p2

# 単体の結果
#LinearRegresssion MSE 2495554898.668321
#SVR MSE 7842006843.379719
#決定木 MSE 2217101574.2512364

# 考察(上回るパターン1)
# 重回帰と、決定木のブレンディングが3種類より優秀 

重回帰、svm MSE 4103105570.4928856
重回帰、決定木 MSE 1857217416.7137222
svm、決定木 MSE 3249053550.3458395


In [4]:
p =1/3
p2 = 1/3
p3 = 1/3

#  ブレンディング：svm, 決定木、重回帰
pred = pred_svm*p + pred_tree *p2 + pred_linear*p3
print("svm、決定木、重回帰 MSE {}".format(mean_squared_error(pred, y_test)))

# 単体の結果
# LinearRegresssion MSE 2495554898.668321
# SVR MSE 7842006843.379719
# 決定木 MSE 2217101574.2512364

# 考察
# 平均値で所得すると、svmと比べて大分良くなる
# linearや決定木よりスコアが好ましい訳ではないが、それに近しいスコアが出力されている

svm、決定木、重回帰 MSE 2701398910.915856


In [5]:
p =0.2
p2 = 0.5
p3 = 0.3

#  ブレンディング：svm, 決定木、重回帰
pred = pred_svm*p + pred_tree *p2 + pred_linear*p3
print("svm、決定木、重回帰 MSE {}".format(mean_squared_error(pred, y_test)))

# 単体の結果
# LinearRegresssion MSE 2495554898.668321
# SVR MSE 7842006843.379719
# 決定木 MSE 2217101574.2512364

# 考察(上回るパターン2)
# SVMよりは確実に良い。linearより優れており、決定木とほぼ同じスコア
# 現在のスコアだけで判断できないが、汎化性を考えるとこちらの方が優秀そう

svm、決定木、重回帰 MSE 2158189798.039097


In [6]:
p =0.7
p2 = 0.3

# ブレンディング：重回帰、svm
pred = pred_linear*p + pred_svm *p2
print("重回帰、svm MSE {}".format(mean_squared_error(pred, y_test)))

# ブレンディング：重回帰、決定木
pred = pred_linear*p + pred_tree *p2
print("重回帰、決定木 MSE {}".format(mean_squared_error(pred, y_test)))

# ブレンディング：svm, 決定木
pred = pred_svm*p + pred_tree *p2
print("svm、決定木 MSE {}".format(mean_squared_error(pred, y_test)))

#  ブレンディング：svm, 決定木、重回帰
pred = pred_svm*p + pred_tree *p2

# 単体の結果
# LinearRegresssion MSE 2495554898.668321
# SVR MSE 7842006843.379719
# 決定木 MSE 2217101574.2512364

# ブレンド5:5
# 重回帰、svm MSE 4103105570.4928856
# 重回帰、決定木 MSE 1916348710.9180086
# svm、決定木 MSE 3407691598.4304748

# 考察(上回るパターン3)
# ブレンドの比率を変更したところ、結果は全て悪化
# 汎化性だけを考慮すると、定かではないが。。。

重回帰、svm MSE 3204323229.6355877
重回帰、決定木 MSE 1996335496.476371
svm、決定木 MSE 4662484393.246541


# 【問題2】バギングのスクラッチ実装
バギング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。


バギングとは
バギングは入力データの選び方を多様化する方法です。学習データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ ブートストラップサンプル ）を作り出します。それらによってモデルをN個学習し、推定結果の平均をとります。ブレンディングと異なり、それぞれの重み付けを変えることはありません。


sklearn.model_selection.train_test_split — scikit-learn 0.21.3 documentation


scikit-learnのtrain_test_splitを、shuffleパラメータをTrueにして使うことで、ランダムにデータを分割することができます。これによりブートストラップサンプルが手に入ります。


推定結果の平均をとる部分はブースティングと同様の実装になります。

In [7]:
import numpy as np

# バギング
baging_iter = 3
score = np.zeros(baging_iter)

# 訓練データとテストデータを分割する
X_train, X_test, y_train, y_test = train_test_split(X_scaler, y, test_size=0.2) 

# for分で訓練データの中で、更に分割する
for n in range(baging_iter):
    X_train_div, X_valid, y_train_div, y_valid = train_test_split(X_train, y_train, test_size=0.2)
    model_tree = DecisionTreeRegressor()
    model_tree.fit(X_train_div, y_train_div)
    pred = model_tree.predict(X_test)
    score[n] = mean_squared_error(pred, y_test)
    
    # nが最後の値ならば、平均を求める
    if n == (baging_iter -1):
        score_mean = np.mean(score)
        print(score_mean)

3300638391.428304


# 【問題3】スタッキングのスクラッチ実装
スタッキング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。


スタッキングとは
スタッキングの手順は以下の通りです。最低限ステージ0とステージ1があればスタッキングは成立するため、それを実装してください。まずは 
K
0
=
3
,
M
0
=
2
 程度にします。


《学習時》


（ステージ 
0
 ）


学習データを 
K
0
 個に分割する。
分割した内の 
(
K
0
−
1
)
 個をまとめて学習用データ、残り 
1
 個を推定用データとする組み合わせが 
K
0
 個作れる。
あるモデルのインスタンスを 
K
0
 個用意し、異なる学習用データを使い学習する。
それぞれの学習済みモデルに対して、使っていない残り 
1
 個の推定用データを入力し、推定値を得る。（これをブレンドデータと呼ぶ）
さらに、異なるモデルのインスタンスも 
K
0
 個用意し、同様のことを行う。モデルが 
M
0
 個あれば、 
M
0
 個のブレンドデータが得られる。

（ステージ 
n
 ）


ステージ 
n
−
1
 のブレンドデータを
M
n
−
1
 次元の特徴量を持つ学習用データと考え、 
K
n
 個に分割する。以下同様である。

（ステージ 
N
 ）＊最後のステージ


ステージ 
N
−
1
 の 
M
N
−
1
 個のブレンドデータを
M
N
−
1
 次元の特徴量の入力として、1種類のモデルの学習を行う。これが最終的な推定を行うモデルとなる。

《推定時》


（ステージ 
0
 ）


テストデータを 
K
0
×
M
0
 個の学習済みモデルに入力し、
K
0
×
M
0
 個の推定値を得る。これを 
K
0
 の軸で平均値を求め 
M
0
 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ 
n
 ）


ステージ 
n
−
1
 で得たブレンドテストを 
K
n
×
M
n
 個の学習済みモデルに入力し、
K
n
×
M
n
 個の推定値を得る。これを 
K
n
 の軸で平均値を求め 
M
0
 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ 
N
 ）＊最後のステージ


ステージ 
N
−
1
 で得たブレンドテストを学習済みモデルに入力し、推定値を得る。

In [9]:
from sklearn.model_selection import KFold

y= y[:, np.newaxis]
X_train, X_test, y_train, y_test = train_test_split(X_scaler, y, test_size=0.2) 

# 再度fitさせる特徴量と、validデータの推定値を出力する
def stacking(clf, X_train, y_train, X_test, k):
    kf = KFold(n_splits=k,shuffle=True)
    pred_list =[]
    preds_test_list = []
    va_index = []
    
    # k-fold
    for train_index, test_index in kf.split(X_train):
        X_train_div, X_valid =  X_train[train_index],  X_train[test_index]
        y_train_div, y_valid = y_train[train_index], y_train[test_index] 
        print(X_train_div.shape,X_valid.shape, y_train_div.shape, y_valid.shape)
        
        # モデルに学習させ、予測値とインデックスを保存
        clf.fit(X_train_div, y_train_div)
        pred = clf.predict(X_valid)
        pred_list.append(pred)
        pred_test = clf.predict(X_test)
        preds_test_list.append(pred_test)
        va_index.append(test_index)
    print(preds_test_list[0].shape)
    
    # 　バリデーションデータに対する予測値をマージし、その後元の順序に戻す
    va_index = np.concatenate(va_index)
    #print(va_index)
    pred_list = np.concatenate(pred_list, axis=0) # 特徴量だから？
    order = np.argsort(va_index)
    pred_train = pred_list[order]
    
    # validデータに対する予測値の平均を取る
    #　print(preds_test_list)    
    preds_test_list = np.mean(preds_test_list, axis=0) # こっちは不要マージ (292,1)が出力される　kfoldのfor文では(292,1)が３回繰り返されている
    return pred_train, preds_test_list

In [21]:
pred_train, pred_test = stacking(model_linear, X_train,y_train, X_test, k=3)
print(pred_train.shape)# 訓練データをvalidで予測したもの（次の特徴量）→再度fitさせるもの
print(pred_test.shape)# 訓練データでtestを予測したもの

import warnings 
warnings.simplefilter('ignore')
# 訓練用データと、predで出力された値をsvm、linearで所得する
pred_train_linear, pred_test_linear = stacking(model_linear, X_train,y_train, X_test, k=3)
pred_train_svm, pred_test_svm = stacking(model_svm, X_train,y_train, X_test, k=3)

(778, 2) (390, 2) (778, 1) (390, 1)
(779, 2) (389, 2) (779, 1) (389, 1)
(779, 2) (389, 2) (779, 1) (389, 1)
(292, 1)
(1168, 1)
(292, 1)
(778, 2) (390, 2) (778, 1) (390, 1)
(779, 2) (389, 2) (779, 1) (389, 1)
(779, 2) (389, 2) (779, 1) (389, 1)
(292, 1)
(778, 2) (390, 2) (778, 1) (390, 1)
(779, 2) (389, 2) (779, 1) (389, 1)
(779, 2) (389, 2) (779, 1) (389, 1)
(292,)


In [39]:
# 1層目評価
print(mean_squared_error(pred_train_linear, y_train))
print(mean_squared_error(pred_train_svm, y_train))

# 予測値を特徴量としてデータフレームを作成する
train_x_2 = pd.DataFrame({'pred_linear':pred_train_linear.reshape(-1), 'pred_svm': pred_train_svm.reshape(-1)})
test_x_2 = pd.DataFrame({'pred_linear':pred_test_linear.reshape(-1), 'pred_svm':pred_test_svm.reshape(-1)})

train_x_2 = np.array(train_x_2)

# ２層目
pred_train_tree, pred_test_tree = stacking(model_tree, train_x_2,y_train, test_x_2, k=3)

print(mean_squared_error(pred_train_tree, y_train))

# 総論
#　決定木を最後のモデルに使った結果、悪化している
# 元々決定木単体のスコアが良いので、他のスコアの悪い物を混ぜている分低くなっているだけだと思う。

2308422094.396909
6685585201.941444
(778, 2) (390, 2) (778, 1) (390, 1)
(779, 2) (389, 2) (779, 1) (389, 1)
(779, 2) (389, 2) (779, 1) (389, 1)
(292,)
3847599546.224315
